In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn

In [7]:
df = pd.read_csv('Cleaned_Data.csv')

In [8]:
df = df.dropna(subset=['Writer'])

In [9]:
import re
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stopwords_list = stopwords.words('english')

stopwords_list
stopwords_list.append(" ")
stopwords_list.append("")
stopwords_list.extend([chr(i) for i in range(ord('a'), ord('z') + 1)])

# Define a function to split text into words and remove specified words
def split_into_words_and_remove_stopwords(text, stopwords):
    # Split text using various separators (space, period, comma)
    words = re.split(r'\s+|[,\.:!?"&]', text)
    # Remove specified stopwords from the list
    words = [word.lower() for word in words if word.lower() not in stopwords]
    return words


# Apply the function to the "storyline" column
df['Storyline'] = df['Storyline'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))
df['Genre'] = df['Genre'].apply(lambda x: split_into_words_and_remove_stopwords(x, stopwords_list))
df['Stars'] = df['Stars'].apply(lambda x: x.split(','))
df['Writer'] = df['Writer'].apply(lambda x: x.split(','))

# Display the DataFrame
df['Certificate'].unique()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kingk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array(['PG-13', 'Unrated', 'PG', 'R', 'G', 'TV-14', 'TV-MA', 'TV-PG',
       'NC-17', 'TV-Y7', 'TV-G', 'C13', 'Approved', 'C18', 'C16', 'C12',
       'MA-17', 'TV-Y7-FV', 'AO', 'TV-Y', 'M', '(Banned)', 'C6', 'PG-12',
       'GP', 'U', 'C7', 'C15', 'AL', 'C9'], dtype=object)

In [23]:
df_cat = df[['Certificate', 'Director', 'Rating', 'Rating Count', 'Genre']]
df_cat['Drama'] = ['drama' in value for value in df_cat['Genre'].values]
df_cat['Certificate'].unique()

C:\Users\kingk\AppData\Local\Temp\ipykernel_5824\1224251463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cat['Drama'] = ['drama' in value for value in df_cat['Genre'].values]


array(['PG-13', 'Unrated', 'PG', 'R', 'G', 'TV-14', 'TV-MA', 'TV-PG',
       'NC-17', 'TV-Y7', 'TV-G', 'C13', 'Approved', 'C18', 'C16', 'C12',
       'MA-17', 'TV-Y7-FV', 'AO', 'TV-Y', 'M', '(Banned)', 'C6', 'PG-12',
       'GP', 'U', 'C7', 'C15', 'AL', 'C9'], dtype=object)

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Xử lý đặc trưng trước khi tách ra thành tập huấn luyện và tập kiểm tra
numeric_features = ['Rating', 'Rating Count']
categorical_features = ['Certificate', 'Director']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Tách ra thành X (đặc trưng) và y (nhãn)
X = df_cat[['Certificate', 'Director', 'Rating', 'Rating Count']]
y = df_cat['Drama']

# Áp dụng xử lý đặc trưng cho toàn bộ dữ liệu
X_preprocessed = preprocessor.fit_transform(X)

# Chia thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)

# Tạo pipeline cho mô hình
model = Pipeline(steps=[
    ('classifier', LinearSVC())
])

# Huấn luyện mô hình trên tập huấn luyện
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Accuracy: 0.6896331058020477
Precision: 0.7065826330532213
Recall: 0.7658444022770399
F1-score: 0.735020943361865


C:\Users\kingk\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
